## ETL: Preparación de datasets

Extracción:  
En primera instancia obtendremos los datasets en crudo del siguiente repositorio: https://github.com/soyHenry/fe-ct-pimlops2  
O de la siguiente página: https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5  
De estos links obtenremos dos archivos en formato .csv.  
Veremos paso a paso una guía breve para realizar una limpieza y obtener los datasets aptos para subir a un repositorio de github y que puedan ser consumidos por nuestra API a través de Render.
  
Una vez los archivos en nuestra computadora, procedemos a importar los archivos y a comenzar la transformación.  
  
Transformación:

In [1]:
import pandas as pd
import ast
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

# Cargar el CSV en un DataFrame
df_movies = pd.read_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1_files/Datasets/movies_dataset.csv')
df_credits =pd.read_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1_files/Datasets/credits.csv')

In [2]:
#Visualizamos las dimensiones del dataframe movies, sus tipos, sus nulos y su peso.
print(df_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
df_movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [5]:
#Visualizamos las dimensiones del dataframe credits, sus tipos, sus nulos y su peso.
print(df_credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


In [6]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Tenemos una lista con objetivos que cumplir en esta etapa de desarrollo del proyecto:

1) Desanidado de columnas:  
Identificar qué columnas están anidadas: belongs_to_collection, production_companies, genres, production_countries y spoken language. También será necesario desanidar el dataset Credits.

2) Tratamiento de valores nulos:  
Nulos cambiados por cero: Revenue y Budget.
Nulos a eliminar: Release.

3) Tratamiento de fechas:  
Formato pedido: AAAA-mm-dd  
  
4) Creación de columnas nuevas:  
Creación de columna release_year: año de la fecha de estreno.  
Creación de columna return: Retorno de inversión; revenue / budget (si no hay datos disponibles para calcularlo, deberá tomar el valor 0)

5) Eliminación de columnas:  
Video, imdb_id, adult, original_title, poster_path y homepage.

In [2]:
# 1) DESANIDADO DE COLUMNAS:

# Probamos técnicas para desanidado con una sola columna en un principio.
column_name = 'genres'

# Definimos una función que maneje NaN y cadenas mal formateadas
def safe_literal_eval(val):
    try:
        if pd.isna(val):
            return []
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Aplicamos la función a la columna para convertirla a listas de diccionarios
df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

# Extraemos las IDs y Names en nuevas columnas
df_movies['genres_id'] = df_movies[column_name].apply(lambda x: [d['id'] for d in x])
df_movies['genres_name'] = df_movies[column_name].apply(lambda x: [d['name'] for d in x])

# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['genres'], inplace=True)

# Guardamos el resultado en un nuevo archivo CSV
#df_movies.to_csv('movies_dataset_transformado_2.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'")

Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'


In [4]:
# Vemos el progreso del proceso de desanidado sobre las columnas GENRES
df_movies['genres_id']

0        [16, 35, 10751]
1        [12, 14, 10751]
2            [10749, 35]
3        [35, 18, 10749]
4                   [35]
              ...       
45461        [18, 10751]
45462               [18]
45463       [28, 18, 53]
45464                 []
45465                 []
Name: genres_id, Length: 45466, dtype: object

In [5]:
df_movies['genres_name']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres_name, Length: 45466, dtype: object

In [6]:
# Revisamos las columnas que nos quedan por desanidar
df_movies['belongs_to_collection']

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [7]:
df_movies['production_companies']

0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
45461                                                   []
45462               [{'name': 'Sine Olivia', 'id': 19653}]
45463    [{'name': 'American World Pictures', 'id': 6165}]
45464                 [{'name': 'Yermoliev', 'id': 88753}]
45465                                                   []
Name: production_companies, Length: 45466, dtype: object

In [8]:
df_movies['production_countries']

0        [{'iso_3166_1': 'US', 'name': 'United States o...
1        [{'iso_3166_1': 'US', 'name': 'United States o...
2        [{'iso_3166_1': 'US', 'name': 'United States o...
3        [{'iso_3166_1': 'US', 'name': 'United States o...
4        [{'iso_3166_1': 'US', 'name': 'United States o...
                               ...                        
45461               [{'iso_3166_1': 'IR', 'name': 'Iran'}]
45462        [{'iso_3166_1': 'PH', 'name': 'Philippines'}]
45463    [{'iso_3166_1': 'US', 'name': 'United States o...
45464             [{'iso_3166_1': 'RU', 'name': 'Russia'}]
45465     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
Name: production_countries, Length: 45466, dtype: object

In [9]:
df_movies['spoken_languages']

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45461               [{'iso_639_1': 'fa', 'name': 'فارسی'}]
45462                    [{'iso_639_1': 'tl', 'name': ''}]
45463             [{'iso_639_1': 'en', 'name': 'English'}]
45464                                                   []
45465             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45466, dtype: object

In [12]:
# Repetimos el proceso con las otras columnas
columns_to_process = {
    'belongs_to_collection': ['id', 'name', 'poster_path', 'backdrop_path'],
    'production_companies': ['id', 'name'],
    'production_countries': ['iso_3166_1', 'name'],
    'spoken_languages': ['iso_639_1', 'name']
}

# Procesamos cada columna en la lista
for column_name, keys in columns_to_process.items():
    # Convertimos la columna a listas de diccionarios
    df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

    # Verificamos si hay valores inesperados en la columna
    invalid_values = df_movies[~df_movies[column_name].apply(lambda x: isinstance(x, list))][column_name]
    if not invalid_values.empty:
        print(f"Valores inesperados en la columna {column_name}:")
        print(invalid_values)
    
    # Extraemos las claves correspondientes y crear nuevas columnas
    for key in keys:
        df_movies[f'{column_name}_{key.upper()}'] = df_movies[column_name].apply(lambda x: [d.get(key, np.nan) for d in x if isinstance(d, dict)])

# Guardamos el resultado en un nuevo archivo CSV
# df_movies.to_csv('movies_dataset_transformado_final.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_final.csv'")

Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_final.csv'


Hay columnas con valores booleanos! Esto no nos permitirá desarrollar el proceso de desanidado, siendo que son tres registros (contra 45K), procederemos a eliminarlos.

In [11]:
# Lista de índices de las filas que queremos eliminar
indices_to_remove = [19730, 29503, 35587]

# Eliminamos las filas correspondientes
df_movies = df_movies.drop(indices_to_remove)

# ^^^^^^^^^^Repetimos el proceso de la celda anterior! (Después podemos seguir con la siguiente celda)^^^^^^^^^^

In [18]:
# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['belongs_to_collection','production_companies', 'production_countries', 'spoken_languages'], inplace=True)

# Guardamos el DataFrame limpio en un nuevo archivo CSV
#df_cleaned.to_csv('movies_dataset_registros_borrrados.csv', index=False)

print("Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'")

Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'


In [13]:
df_movies

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,belongs_to_collection_ID,belongs_to_collection_NAME,belongs_to_collection_POSTER_PATH,belongs_to_collection_BACKDROP_PATH,production_companies_ID,production_companies_NAME,production_countries_ISO_3166_1,production_countries_NAME,spoken_languages_ISO_639_1,spoken_languages_NAME
0,False,[],30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,[],[],[],[],[3],[Pixar Animation Studios],[US],[United States of America],[en],[English]
1,False,[],65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,[],[],[],[],"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]"
2,False,[],0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,[],[],[],[],"[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English]
3,False,[],16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,[],[],[],[],[306],[Twentieth Century Fox Film Corporation],[US],[United States of America],[en],[English]
4,False,[],0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,[],[],[],[],"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",[US],[United States of America],[en],[English]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,[],0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,...,[],[],[],[],[],[],[IR],[Iran],[fa],[فارسی]
45462,False,[],0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,...,[],[],[],[],[19653],[Sine Olivia],[PH],[Philippines],[tl],[]
45463,False,[],0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,[],[],[],[],[6165],[American World Pictures],[US],[United States of America],[en],[English]
45464,False,[],0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,[],[],[],[],[88753],[Yermoliev],[RU],[Russia],[],[]


In [16]:
# Visualizamos (algunas) de las columnas nuevas generadas
#df_movies['belong_to_colection_id'] #tengamos en cuenta que la mayorias de las columnas de Belong_to_colection están vacias
#df_movies['production_companies_NAME']
df_movies['production_countries_NAME']
#df_movies['spoken_languages_NAME']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45461                        [Iran]
45462                 [Philippines]
45463    [United States of America]
45464                      [Russia]
45465              [United Kingdom]
Name: production_countries_NAME, Length: 45463, dtype: object

In [17]:
# Observamos el dataset de Credits antes de editarlo:
df_credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [18]:
# SEGUIMOS CON EL DATASET CREDITS:
# Lo separaremos en dos datasets; Crew y Cast.

# CREW
# Filtramos filas donde 'crew' tenga longitud mayor a 0
df_valid_crew = df_credits[df_credits['crew'].str.len() > 0]

# Convertimos la columna 'crew' a listas de diccionarios si es necesario
df_valid_crew['crew'] = df_valid_crew['crew'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Expandimos diccionarios en filas separadas
df_crew_expanded = df_valid_crew.explode('crew').reset_index(drop=True)

# Convertimos los diccionarios en columnas
df_crew_expanded = pd.json_normalize(df_crew_expanded['crew'])

In [19]:
df_valid_crew

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [20]:
df_crew_expanded

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
465080,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
465081,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
465082,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
465083,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


In [21]:
# Verificamos la longitud antes de repetir 'id'
expected_length = df_crew_expanded.shape[0]
repeated_ids = df_valid_crew.loc[df_valid_crew.index.repeat(df_valid_crew['crew'].str.len()), 'id'].values

# Si la longitud no coincide, ajustar
if len(repeated_ids) > expected_length:
    repeated_ids = repeated_ids[:expected_length]
elif len(repeated_ids) < expected_length:
    df_crew_expanded = df_crew_expanded.iloc[:len(repeated_ids)]

# Asignamos la columna 'id'
df_crew_expanded['id'] = repeated_ids

# Restablecemos el índice
df_crew_expanded.reset_index(drop=True, inplace=True)

print(df_crew_expanded.head())

                  credit_id department  gender   id         job  \
0  52fe4284c3a36847f8024f49  Directing     2.0  862    Director   
1  52fe4284c3a36847f8024f4f    Writing     2.0  862  Screenplay   
2  52fe4284c3a36847f8024f55    Writing     2.0  862  Screenplay   
3  52fe4284c3a36847f8024f5b    Writing     2.0  862  Screenplay   
4  52fe4284c3a36847f8024f61    Writing     0.0  862  Screenplay   

             name                      profile_path  
0   John Lasseter  /7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg  
1     Joss Whedon  /dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg  
2  Andrew Stanton  /pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg  
3      Joel Cohen  /dAubAiZcvKFbboWlj7oXOkZnTSu.jpg  
4    Alec Sokolow  /v79vlRYi94BZUQnkkyznbGUZLjT.jpg  


In [ ]:
# df_crew_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_crew_expanded.csv', index=False)

In [22]:
# CAST
# Filtramos filas con longitud mayor a 0
df_valid_cast = df_credits[df_credits['cast'].str.len() > 0]

# Convertimos la columna 'cast' a listas de diccionarios si es necesario
df_valid_cast['cast'] = df_valid_cast['cast'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Expandimos diccionarios en filas separadas
df_cast_expanded = df_valid_cast.explode('cast').reset_index(drop=True)

# Convertimos los diccionarios en columnas
df_cast_expanded = pd.json_normalize(df_cast_expanded['cast'])

In [23]:
# Vamos chequeando los progresos del proceso sobre la columna Cast a dataframe:
df_valid_cast

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [34]:
df_cast_expanded

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
564887,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [24]:
# Verificamos la longitud antes de repetir 'id'
expected_length = df_cast_expanded.shape[0]
repeated_ids = df_valid_cast.loc[df_valid_cast.index.repeat(df_valid_cast['cast'].str.len()), 'id'].values

# Si la longitud no coincide, ajustar
if len(repeated_ids) > expected_length:
    repeated_ids = repeated_ids[:expected_length]
elif len(repeated_ids) < expected_length:
    df_cast_expanded = df_cast_expanded.iloc[:len(repeated_ids)]

# Asignamos la columna 'id'
df_cast_expanded['id'] = repeated_ids

# Restablecemos el índice
df_cast_expanded.reset_index(drop=True, inplace=True)

print(df_cast_expanded.head())

   cast_id                character                 credit_id  gender   id  \
0     14.0            Woody (voice)  52fe4284c3a36847f8024f95     2.0  862   
1     15.0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99     2.0  862   
2     16.0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d     2.0  862   
3     17.0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1     2.0  862   
4     18.0              Rex (voice)  52fe4284c3a36847f8024fa5     2.0  862   

            name  order                      profile_path  
0      Tom Hanks    0.0  /pQFoyx7rp09CJTAb932F2g8Nlho.jpg  
1      Tim Allen    1.0  /uX2xVf6pMmPepxnvFWyBtjexzgY.jpg  
2    Don Rickles    2.0  /h5BcaDMPRVLHLDzbQavec4xfSdt.jpg  
3     Jim Varney    3.0  /eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg  
4  Wallace Shawn    4.0  /oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg  


In [ ]:
# df_cast_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_cast_expanded.csv', index=False)

In [25]:
cantidad_nulos = df_movies['revenue'].isnull().sum()
print(f"Cantidad de nulos en la columna 'nombre_columna': {cantidad_nulos}")

Cantidad de nulos en la columna 'nombre_columna': 3


In [26]:
cantidad_nulos = df_movies['budget'].isnull().sum()
print(f"Cantidad de nulos en la columna 'nombre_columna': {cantidad_nulos}")

Cantidad de nulos en la columna 'nombre_columna': 0


In [27]:
# 2) Tratamiento de valores nulos:

# Rellenamos los valores nulos de las columnas 'revenue' y 'budget' con 0
df_movies['revenue'].fillna(0, inplace=True)
# df_movies['budget'].fillna(0, inplace=True) esta linea de código no hace falta dado que tiene cero nulos

# Eliminamos las filas donde 'release_date' es nulo
df_movies = df_movies.dropna(subset=['release_date'])


In [41]:
df_movies['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [42]:
# 3) Tratamiento de fechas:

# Aseguramos que la columna release_date esté en formato datetime
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

# Convertimos la fecha al formato AAAA-mm-dd
df_movies['release_date'] = df_movies['release_date'].dt.strftime('%Y-%m-%d')

In [43]:
df_movies['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [46]:
# 4) Creación de columnas nuevas:

# Creamos una nueva columna release_year extrayendo el año de release_date
df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year

# Convertimos las columnas 'REVENUE' y 'BUDGET' a numéricas, forzando errores a NaN
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

# Creamos la columna 'RETURN' calculando REVENUE / BUDGET
df_movies['return'] = df_movies.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] > 0 else 0, axis=1)

In [47]:
df_movies['release_year']

0        1995.0
1        1995.0
2        1995.0
3        1995.0
4        1995.0
          ...  
45461       NaN
45462    2011.0
45463    2003.0
45464    1917.0
45465    2017.0
Name: release_year, Length: 45463, dtype: float64

In [48]:
# 5) Eliminación de columnas:

# Eliminamos las columnas video, imdb_id, adult, original_title, poster_path y homepage.
df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)



In [18]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,belongs_to_collection_NAME,belongs_to_collection_POSTER_PATH,belongs_to_collection_BACKDROP_PATH,production_companies_ID,production_companies_NAME,production_countries_ISO_3166_1,production_countries_NAME,spoken_languages_ISO_639_1,spoken_languages_NAME,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,373554033.0,81.0,Released,NaN,...,[],[],[],[3],[Pixar Animation Studios],[US],[United States of America],[en],[English],12.451801
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,[],[],[],"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]",4.043035
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,[],[],[],"[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English],0.000000
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[],[],[],[306],[Twentieth Century Fox Film Corporation],[US],[United States of America],[en],[English],5.090760
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,[],[],[],"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",[US],[United States of America],[en],[English],0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991,0.0,104.0,Released,NaN,...,[],[],[],"[7025, 10163, 16323, 38978]","[Westdeutscher Rundfunk (WDR), Working Title F...","[CA, DE, GB, US]","[Canada, Germany, United Kingdom, United State...",[en],[English],0.000000
45462,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011,0.0,360.0,Released,NaN,...,[],[],[],[19653],[Sine Olivia],[PH],[Philippines],[tl],[],0.000000
45463,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003,0.0,90.0,Released,A deadly game of wits.,...,[],[],[],[6165],[American World Pictures],[US],[United States of America],[en],[English],0.000000
45464,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917,0.0,87.0,Released,NaN,...,[],[],[],[88753],[Yermoliev],[RU],[Russia],[],[],0.000000


In [19]:
df_cast_expanded

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,862,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,862,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,862,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,862,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,862,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
562469,12.0,"Dennis (segment ""The Wet Dream"")",55c8d0e7c3a36804a00009ab,2.0,227506,Collin Bernsen,2.0,None
562470,13.0,"Terri (segment ""Tango"")",55c8d103c3a3680408000ad0,0.0,227506,Allison Brooks,3.0,None
562471,14.0,"Lisa (segment ""Within Ten Minutes"")",55c8d126c3a3680408000ad9,0.0,227506,Elizabeth Burr,4.0,None
562472,15.0,"Lulu (segment ""Lulu"")",55c8d14492514173de000864,1.0,227506,Marianna Elliott,5.0,None


In [20]:
df_crew_expanded

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
464309,58e946ce9251414b28024571,Editing,0.0,67758,Editor,Ollie Rainbird,None
464310,52fe4c6c9251416c910f481b,Directing,2.0,67758,Director,Antonin Peretjatko,None
464311,52fe4c6c9251416c910f4821,Camera,0.0,227506,Director of Photography,Simon Roca,None
464312,52fe4c6c9251416c910f4827,Writing,2.0,227506,Writer,Antonin Peretjatko,None
